In [6]:
import logging

import pandas as pd
import json
from asyncio import gather
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
from langchain.chat_models import init_chat_model

In [7]:
logger = logging.getLogger(__name__)
agent_id = "rag"
n_questions = 20

In [8]:
loader = PyPDFLoader(Path("../data/pdfs/pdf24_merged.pdf"))
content = ""
async for page in loader.alazy_load():
    content += page.page_content + "\n"

In [9]:
llm = init_chat_model(
    "llama3.2",  # qwen3:8b
    model_provider="ollama",
    base_url="host.docker.internal",
    temperature=0.2,
)

In [ ]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.utils.json import parse_json_markdown

# Init LLM

# Prompt
prompt = PromptTemplate.from_template(
    """
    Generate **{n_questions}** diverse and realistic question-answer pairs based on the following content.

    Guidelines:
    - Each question must be clearly related to the content.
    - Questions must vary in formality, tone, and complexity, simulating real user behavior.
    - Include both precise technical questions and informal/help-seeking ones (e.g., "Non trovo la configurazione dell'email, mi aiuti?").
    - Answers must be long, descriptive and accurate, based strictly on the provided content.
    - Never use UNICODE characters in the output, only ASCII.
    - When you need to escape double quotes in the output, use a single backslash (e.g., \"example\").
    - After each question-answer pair, specify the exact same words of the part of the content used to generate it.

    Format the response using exactly this JSON structure. It must be valid JSON and parsable in Python. The response must not contain any other text or formatting outside of this JSON structure.:

    ```json
    [
        {{
            "question": "<question>",
            "answer": "<answer>",
            "content_used": "<content used to generate the question>"
        }},
        ...
    ]
    ```
    
    Ensure that the JSON is well-formed and does not contain any trailing commas or syntax errors.
    Do not include any additional text or explanations outside of the JSON structure.

    Content to analyze:

    {chunk}
    """
)

# Parser
parser = JsonOutputParser()

# Chain
chain = prompt | llm  # | parser


# Input doc
# document = "The mitochondrion is the powerhouse of the cell. It generates ATP through respiration."

# Run
output = chain.invoke({"chunk": content, "n_questions": n_questions})
# json_output = parse_json_markdown(output.content)
# print(json_output)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
split = content.index("4. ")
split2 = content.index("9. ")
chunks = [content[:split], content[split:split2], content[split2:]]

In [ ]:
def get_prompt(chunk):
    return f"""
    Generate **{n_questions}** diverse and realistic question-answer pairs based on the following content.

    Guidelines:
    - Each question must be clearly related to the content.
    - Questions must vary in formality, tone, and complexity, simulating real user behavior.
    - Include both precise technical questions and informal/help-seeking ones (e.g., "Non trovo la configurazione dell'email, mi aiuti?").
    - Answers must be long, descriptive and accurate, based strictly on the provided content.
    - Never use UNICODE characters in the output, only ASCII.
    - When you need to escape double quotes in the output, use a single backslash (e.g., \"example\").
    - After each question-answer pair, specify the exact same words of the part of the content used to generate it.

    Format the response using exactly this JSON structure. It must be valid JSON and parsable in Python. The response must not contain any other text or formatting outside of this JSON structure.:

    ```json
    [
        {{
            "question": "<question>",
            "answer": "<answer>",
            "content_used": "<content used to generate the question>"
        }},
        ...
    ]
    ```
    
    Ensure that the JSON is well-formed and does not contain any trailing commas or syntax errors.
    Do not include any additional text or explanations outside of the JSON structure.

    Content to analyze:

    {chunk}
    """


responses = await gather(*[llm.ainvoke(get_prompt(chunk)) for chunk in chunks])

In [43]:
response = json.loads("""
[
    {
        "question": "Come posso configurare una nuova email ordinaria nello studio tramite il portale?",
        "answer": "Per configurare una nuova email ordinaria all'interno del portale, accedi al menu principale e seleziona Configurazioni -> Parametri Email. Clicca sul simbolo '+' per inserire una nuova configurazione. Compila i campi richiesti: 'Descrizione' per identificare l'email, 'Tipo' selezionando 'ORDINARIA', 'Indirizzo' con l'email da configurare, 'Host' scegliendolo dall'elenco dei provider disponibili (se non presente, contattare l’assistenza PLUM), 'Username' che solitamente coincide con l'indirizzo email, e 'Password'. Puoi anche indicare un indirizzo di inoltro per conoscenza se desideri ricevere copie delle email inviate. Infine, salva la configurazione.",
        "content_used": "COME CONFIGUARE LE EMAIL E LA PEC"
    },
    {
        "question": "Non trovo la configurazione dell'email, mi aiuti?",
        "answer": "Per configurare l'email accedi al portale e vai su Configurazioni -> Parametri Email. Da lì premi il simbolo '+' per inserire i parametri richiesti come indirizzo, password, host, e altri. Se il tuo provider non è nell'elenco, contatta l'assistenza PLUM. Una volta completati i campi, salva la configurazione.",
        "content_used": "COME CONFIGUARE LE EMAIL E LA PEC"
    },
    {
        "question": "Cosa fa esattamente il servizio TurboSMTP?",
        "answer": "Il servizio TurboSMTP è un'alternativa all'invio di email tramite il proprio provider. Questo servizio garantisce una percentuale di consegna superiore al 99,7% e fornisce dettagli su data, ora di invio ed eventuali esiti negativi. Per attivarlo, è necessario selezionare l'opzione 'su questo account intendo utilizzare il servizio turboSMTP'. È anche possibile ricevere una copia dell'email su un altro indirizzo attivando il relativo campo e inserendo l'indirizzo nel campo 'Mail inoltro per conoscenza'.",
        "content_used": "COME CONFIGUARE LE EMAIL E LA PEC"
    },
    {
        "question": "Dove si inseriscono le coordinate delle buste da lettere?",
        "answer": "Per inserire le coordinate delle buste da lettere, accedi al menu Configurazioni -> Parametri Buste da Lettere. Nella finestra che si apre puoi specificare le coordinate per i diversi formati disponibili (Americano, Medio, Grande). Dopo aver completato i campi, premi sul simbolo del dischetto per memorizzare le coordinate.",
        "content_used": "COME INSERIRE LE COORDINATE DELLA BUSTA DA LETTERE"
    },
    {
        "question": "C'e' un modo per usare la Dymo per stampare le etichette?",
        "answer": "Sì, la piattaforma permette di stampare le etichette anche con la Dymo. Devi selezionare il formato a 1 pista, che corrisponde a una singola etichetta per riga. L’inserimento delle coordinate avviene tramite il menu Configurazioni -> Parametri Etichette. Da lì compila i campi e premi il simbolo del dischetto per salvare le impostazioni.",
        "content_used": "COME INSERIRE LE COORDINATE DELLE ETICHETTE"
    },
    {
        "question": "Che differenza c'e' tra le 1, 2 e 3 piste delle etichette?",
        "answer": "Le piste rappresentano il numero di etichette affiancate su una riga. 1 pista indica una sola etichetta per riga (es. formato Dymo), 2 piste due etichette per riga, e 3 piste tre etichette. Questa configurazione consente di adattare la stampa al layout dei fogli di etichette utilizzati.",
        "content_used": "COME INSERIRE LE COORDINATE DELLE ETICHETTE"
    },
    {
        "question": "Come posso configurare un bollettino 123?",
        "answer": "Per configurare i bollettini 123 accedi al menu principale, vai su Configurazioni -> Parametri Bollettino 123. Si apre una finestra in cui inserire i parametri, simili a quelli delle email. Dopo aver compilato tutti i campi (descrizione, tipo, indirizzo, host, username, password, ecc.), premi il pulsante SALVA per memorizzare le informazioni.",
        "content_used": "COME IMPOSTARE LE COORDINATE DEI BOLLETTINI 123"
    },
    {
        "question": "Dove si salvano i testi personalizzati per le email?",
        "answer": "I testi personalizzati si configurano dal menu Configurazioni -> Testi personalizzati. Dopo aver cliccato sul pulsante '+' nella parte superiore della finestra, puoi selezionare il riferimento (es. Email), inserire un titolo e il contenuto del testo. Questo testo potrà poi essere richiamato durante l’invio di email, pec o solleciti.",
        "content_used": "COME CONFIGUARE I TESTI PERSONALIZZATI"
    },
    {
        "question": "Posso scrivere testi diversi per ogni sollecito?",
        "answer": "Sì, la piattaforma permette di impostare testi differenti per ciascun destinatario nei solleciti. È possibile associare testi personalizzati a ogni invio selezionando 'Sollecito' nel menu Configurazioni -> Testi personalizzati. Così puoi differenziare il contenuto per ogni destinatario invece di inviare lo stesso testo a tutti.",
        "content_used": "COME CONFIGUARE I TESTI PERSONALIZZATI"
    },
    {
        "question": "Come cambio i dati del mittente sulle buste?",
        "answer": "Per modificare i dati del mittente da stampare sulle buste, vai su Configurazioni -> Dati Mittente. Da lì puoi scegliere se usare i dati dell’Amministratore, del Condominio o dello Studio. Dopo aver inserito i dati, seleziona il mittente predefinito. Durante l'invio, i dati verranno usati automaticamente, ma puoi comunque modificarli prima della spedizione.",
        "content_used": "COME IMPOSTARE I DATI DEL MITTENTE"
    },
    {
        "question": "Come faccio a spedire un sollecito via posta ordinaria?",
        "answer": "La piattaforma PLUM ti consente di inviare documenti (come solleciti) anche tramite posta ordinaria. I dati arrivano dal portale Lemon attraverso il pulsante PLUM presente nella finestra dei solleciti. Una volta dentro PLUM, nella sezione SPEDIZIONI trovi i destinatari e i documenti allegati. Compila il campo CONTENUTO per descrivere la spedizione. I dati del mittente sono presi da quelli predefiniti ma puoi modificarli prima dell’invio.",
        "content_used": "COME IMPOSTARE I DATI PER LA SPEDIZIONE"
    },
    {
        "question": "Come si aggiornano i miei dati personali?",
        "answer": "Per aggiornare i tuoi dati personali, dalla pagina iniziale premi il pulsante Profilo. Verrà visualizzata una finestra dove l’amministratore può modificare i propri dati. Dopo aver effettuato le modifiche, è fondamentale premere il pulsante SALVA per applicare i cambiamenti.",
        "content_used": "COME MODIFICA I DATI DEL PROFILO"
    },
    {
        "question": "Quanto costano i documenti digitali e come si comprano?",
        "answer": "I documenti digitali costano 0,30 euro + IVA ciascuno e si acquistano dalla pagina ORDINI. Dopo aver cliccato su '+', inserisci la quantità desiderata e i dati per la fattura. Premi SALVA, poi conferma l’ordine premendo il carrello verde e la stampante blu. Dopo il pagamento, i documenti vengono accreditati automaticamente sul contatore visibile nella pagina iniziale o in Estratto Conto.",
        "content_used": "COSA SONO I DOCUMENTI DIGITALI E COME ACQUISTARLI"
    },
    {
        "question": "Ho finito i crediti per i documenti digitali. Come faccio?",
        "answer": "Accedi alla pagina ORDINI, premi '+', specifica la quantità di documenti digitali da acquistare e inserisci i dati fiscali. Dopo aver premuto SALVA, conferma l’ordine tramite il carrello verde e poi stampa il bollettino 896. Una volta pagato, il sistema accredita automaticamente i nuovi crediti aggiornando il contatore.",
        "content_used": "COSA SONO I DOCUMENTI DIGITALI E COME ACQUISTARLI"
    },
    {
        "question": "Come genero i bollettini TD 123 da Lemon?",
        "answer": "Accedi a Lemon, seleziona il condominio, poi vai su Comunicazioni -> Avvisi-Bollettini-Mav e scegli Bollettini postali TD 123 Matrice. Dopo aver selezionato le rate, compila i campi e premi STAMPA per la prova. Se tutto e' corretto, togli la spunta 'stampa di prova' per abilitare il pulsante PLUM. I dati verranno trasferiti a PLUM, dove potrai procedere con la spedizione.",
        "content_used": "COME GENERARE I BOLLETTINI POSTALI TD 896 O TD 123"
    },
    {
        "question": "Perche' non vedo il pulsante PLUM attivo?",
        "answer": "Il pulsante PLUM rimane disattivato finche' la spunta 'Attenzione, stampa di prova ...' e' attiva. Una volta verificati i dati nella stampa di prova, rimuovi la spunta per abilitare il pulsante PLUM e trasferire i dati alla piattaforma PLUM.",
        "content_used": "COME GENERARE I BOLLETTINI POSTALI TD 896 O TD 123"
    },
    {
        "question": "Dove vado per scaricare i dati da Lemon su PLUM?",
        "answer": "Accedi a www.plumpost.it con le credenziali, poi nella sezione SPEDIZIONI premi sul cilindro celeste. Questo consente di scaricare i dati trasmessi da Lemon, inclusi i bollettini postali generati.",
        "content_used": "COME GENERARE I BOLLETTINI POSTALI TD 896 O TD 123"
    },
    {
        "question": "Serve salvare ogni configurazione che inserisco?",
        "answer": "Sì, dopo ogni configurazione (email, buste, etichette, mittente, ecc.) è necessario premere il pulsante SALVA o il simbolo del dischetto per memorizzare i dati inseriti, altrimenti le impostazioni non verranno applicate.",
        "content_used": "COME CONFIGUARE LE EMAIL E LA PEC"
    },
    {
        "question": "Il campo 'Mail inoltro per conoscenza' e' obbligatorio?",
        "answer": "No, il campo 'Mail inoltro per conoscenza' è facoltativo e va compilato solo se si attiva la spunta 'su servizio turboSMTP desidero ricevere la copia email per conoscenza'. In tal caso bisogna indicare un indirizzo email valido dove ricevere le copie.",
        "content_used": "COME CONFIGUARE LE EMAIL E LA PEC"
    }
]
""")

In [44]:
golden_set = pd.DataFrame(response).reset_index(names=["id"])

In [ ]:
# contents = [
#     r.content.replace("```json", "").replace("```", "").strip() for r in responses
# ]

question_answer_pairs = [json.loads(r.content)["questions"] for r in responses]

# c1 = json.loads(contents[0])["questions"]
# i = contents[1].rfind("},\n    }\n]")
# c = contents[1][:i] + contents[1][i:].replace("},", "}")
# c2 = json.loads(c)["questions"]
# question_answer_pairs = [c1, c2]

question_answer_pairs = sum(question_answer_pairs, [])

print(question_answer_pairs)

In [ ]:
golden_set = pd.DataFrame(question_answer_pairs).reset_index(names=["id"])

In [ ]:
golden_set

In [46]:
golden_set.to_csv("data/generated_golden_set.csv", index=False)